In [ ]:
import functions_framework
import pandas as pd
from sqlalchemy import create_engine 
import requests
from timezonefinder import TimezoneFinder #To find timezone
import pytz
from datetime import datetime #, timedelta
#Please create python file to save your sql password and user API Key
from safe_key import safe_key

@functions_framework.http
def weather_data(request):
  connection_string = connection()
  #Retrieving information from sql cities_info basic information
  cities_info_from_sql = pd.read_sql("cities_info", con = connection_string)
  
  #Update weather data
  WeatherData(cities_info_from_sql, connection_string, "metric")
  return "Data successfully added"

def connection():
  schema = "gans_cloud"
  host = safe_key["Host_IP"]
  user = "root"
  port = 3306
  password = safe_key["SQL_cloud_password"]

  db = f"mysql+pymysql://{user}:{password}@{host}:{port}/{schema}"
  return db

#Sending python data to sql table
def send_data(df, connection_string, sql_table):
  return df.to_sql(name = sql_table,
                  con = connection_string,
                  if_exists ='append',
                  index = False) #Ensures the DataFrame’s index is not written as a separate column in the database table.


def WeatherData(cities_info_from_sql, connection_string, temp_unit): 
  
  #store all 'values' belong to key 'list' (key:value)
  weather_data = []

  #This loop iterate over cities table in rowwise maanner
  for row in cities_info_from_sql.itertuples(index=False):
  
    #Extracts city details
    city_name = row.City
    city_id = row.City_id
    latitude = row.Latitude  
    longitude = row.Longitude  
    
    #Finds the city's timezone using the TimezoneFinder library
    tf = TimezoneFinder()
    timezone_str = tf.timezone_at(lng=longitude, lat=latitude) or "UTC"
    city_timezone = pytz.timezone(timezone_str)

    # Get current UTC time
    utc_time = datetime.now(pytz.utc)

    # Convert UTC time to the city's local timezone
    city_time = utc_time.astimezone(city_timezone).strftime("%Y-%m-%d %H:%M:%S")
    city_time_clean = pd.to_datetime(city_time)

    #Define base url
    base_url = "https://api.openweathermap.org/data/2.5/forecast"

    #Define parameters
    parameters = {"q" : city_name,
                  "appid" : "8d2bfd780b2809aead1ab12b68f10f11",#safe_key["API_key"],
                  "units" : temp_unit}
    
    response = requests.get(base_url, params=parameters)
    
    if response.status_code != 200:
        print(f"Failed to fetch weather data for {city_name}. HTTP Status: {response.status_code}")
        continue
    
    try:
        weather = response.json()
    except requests.exceptions.JSONDecodeError:
        print(f"Invalid JSON response for weather data for {city_name}")
        continue
    

    for count in range(weather['cnt']): #cnt - A number of timestamps returned in the API response

          
      all_data = weather['list'][count]

      #Time of data forecasted, ISO, UTC
      date_time = all_data['dt_txt']
      date_time_clean = pd.to_datetime(date_time)

      # actual feels like temperature in degree celcius
      temperature = all_data['main']['feels_like']

      #weather_condition within the group
      weather_condition = all_data['weather'][0]['description']

      #cloudiness in percentage
      cloudiness = all_data['clouds']['all']

      #wind_speed in meter/sec
      wind_speed = all_data['wind']['speed']

      #pop-Probability_of_Perception values between 0 & 1. Indicated probabiltiy of rain
      pop = all_data['pop']

      #Part of the day (n-night, d-day)
      pod = all_data['sys']['pod']

      weather_data.append({"City_id" : city_id,
                          "Forecast_utc_time" : date_time_clean,
                          "City_time": city_time_clean,
                          "Part_of_day" : pod,
                          "Weather_condition" : weather_condition,
                          "Cloudiness_percent" : cloudiness,
                          "Temp_deg" : temperature,
                          "Wind_speed_M_per_sec" : wind_speed,
                          "Rain_probability_percent" : pop * 100
                          })
  
  weather_df = pd.DataFrame(weather_data)

  #To drop duplicate entries
  existing_weather_df = pd.read_sql("cities_weather", con = connection_string)

  merged_df = weather_df.merge(existing_weather_df, on=["City_id","Forecast_utc_time"], how="left", indicator=True, suffixes= ("", "_existing"))

  #Its create merge column automatically bez of indicator=True
  weather_to_db = merged_df.loc[merged_df["_merge"] == "left_only", weather_df.columns]  

  send_data(weather_to_db, connection_string, "cities_weather")
  
  return

In [3]:
from flask import Request
import json
request_data = {}
request = Request.from_values(data=json.dumps(request_data))

response = weather_data(request)
print(response)

Data successfully added


In [1]:
import os
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello_world():
    return "Hello, World!"

if __name__ == "__main__":
    port = int(os.environ.get("PORT", 8080))
    app.run(host="0.0.0.0", port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.0.251:8080
Press CTRL+C to quit
127.0.0.1 - - [06/Dec/2024 16:05:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2024 16:05:26] "GET /favicon.ico HTTP/1.1" 404 -
192.168.0.251 - - [06/Dec/2024 16:05:41] "GET / HTTP/1.1" 200 -
192.168.0.251 - - [06/Dec/2024 16:05:42] "GET /favicon.ico HTTP/1.1" 404 -
